# Notebook for End to End testing in the MID PSI

- DishLMC in the loop
- SPFRx in the loop
- TMC/SDP/CSP in the loop
- Receives and plots the visibilities to SDP

In [ ]:
import json
import os
from time import sleep

from tango import Database, DevFailed, DeviceProxy

print("Setting up parameters\n")

PSI_NAMESPACE = "ci-ska-mid-psi-1377844469-jaredmda"

TANGO_HOST = f"databaseds-tango-base.{PSI_NAMESPACE}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

RECEPTORS = ["SKA001"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/psi/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/psi/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# MCS/Talon setup
target_board = [1]
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "psi/cbf")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")

print("Links for Taranta and QA Display\n")
print(f"https://142.73.34.170/{PSI_NAMESPACE}/signal/display/")
print(f"https://142.73.34.170/{PSI_NAMESPACE}/taranta/devices")

### MCS Deployer Setup

In [ ]:
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

if any(i > 4 for i in target_board):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_board = list(map(lambda x: x - 4, target_board))
else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

In [ ]:
!kubectl cp $HW_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

In [ ]:
deployer_tango.targetTalons = target_board
print(deployer_tango.targetTalons)

In [ ]:
deployer_tango.generate_config_jsons()

In [ ]:
deployer_tango.set_timeout_millis(300000)
try:
    deployer_tango.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)

In [ ]:
deployer_tango.configure_db()

### Set Up and Configure Device Proxies 

In [ ]:
print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### Load the Dish Vcc Config / Init Sys Params

NOTE: RUN TWICE

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

##### Use This to Check if Config is Set Before Proceeding

In [ ]:
print(tmc_central_node.isDishVccConfigSet)

### Turn the Telescope On

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

while int(tmc_central_node.telescopeState) != 0:
    print("Waiting for Telescope to come ON")
    sleep(5)

print("YAY Telecope is ON")

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

### Assign Resources

In [ ]:
print(sdp_subarray.state())

sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")
print(f"Assign Resource: {assign_resources_json}")

In [ ]:
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")

In [ ]:
#Massage this to work in the PSI
import time
dish_manager = DeviceProxy("mid-dish/dish-manager/ska001")

dish_manager.slew([181.0,31.0])

while dish_manager.achievedpointing[1] != 181.0 or dish_manager.achievedpointing[2] != 31.0:
  time.sleep(1)

print("Done slewing")

### Configure Scan

##### Wait for SDP Pods Before Proceeding

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))

sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")
print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")

### Scan

DO NOT USE FOR NOW

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

### END SCAN


DO NOT USE FOR NOW SINCE TMC SUBARRAY IS STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_subarray.EndScan()

sleep(2)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### END

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_subarray.End()

sleep(2)
print(f"\nTMC Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Release Resources

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()

sleep(2)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")